# Using Amazon Elastic Inference with a pre-trained TensorFlow Serving model from NGC on SageMaker

This notebook demonstrates how to enable and use Amazon Elastic Inference with our predefined SageMaker TensorFlow Serving containers.

Amazon Elastic Inference (EI) is a resource you can attach to your Amazon EC2 instances to accelerate your deep learning (DL) inference workloads. EI allows you to add inference acceleration to an Amazon SageMaker hosted endpoint or Jupyter notebook for a fraction of the cost of using a full GPU instance. For more information please visit: https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html

This notebook's main objective is to show how to create an endpoint, backed by an Elastic Inference, to serve our pre-trained TensorFlow Serving model for predictions. With a more efficient cost per performance, Amazon Elastic Inference can prove to be useful for those looking to use GPUs for higher inference performance at a lower cost.

1. [The model](#The-model)
1. [Setup role for SageMaker](#Setup-role-for-SageMaker)
1. [Load the TensorFlow Serving Model on Amazon SageMaker using Python SDK](#Load-the-TensorFlow-Serving-Model-on-Amazon-SageMaker-using-Python-SDK)
1. [Deploy the trained Model to an Endpoint with EI](#Deploy-the-trained-Model-to-an-Endpoint-with-EI)
    1. [Using EI with a SageMaker notebook instance](#Using-EI-with-a-SageMaker-notebook-instance)
    1. [Invoke the Endpoint to get inferences](#Invoke-the-Endpoint-to-get-inferences)
    1. [Delete the Endpoint](#Delete-the-Endpoint)

If you are familiar with SageMaker and already have a trained model, skip ahead to the [Deploy the trained Model to an Endpoint with an attached EI accelerator](#Deploy-the-trained-Model-to-an-Endpoint-with-an-attached-EI-accelerator)

For this example, we will use the SageMaker Python SDK, which helps deploy your models to train and host in SageMaker. In this particular example, we will be interested in only the hosting portion of the SDK.

1. Set up our pre-trained model for consumption in SageMaker
2. Host the model in an endpoint with EI
3. Make a sample inference request to the model
4. Delete our endpoint after we're done using it


## The model

The pre-trained model we will be using for this example is a NGC ResNet-50 model from the [NGC Model repository](https://ngc.nvidia.com/catalog/models/nvidia:rntf_fp16/files). For more information in regards to deep residual networks, please check [here](https://github.com/tensorflow/models/tree/master/official/resnet). It isn't a requirement to train our model on SageMaker to use SageMaker for serving our model.

SageMaker expects our models to be compressed in a tar.gz format in S3. Thankfully, our model already comes in that format. The predefined TensorFlow Serving containers use REST API for handling inferences, for more informationm, please see [Deploying to TensorFlow Serving Endpoints](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

To host our model for inferences in SageMaker, we need to first upload the SavedModel to S3. This can be done through the AWS console or AWS command line.

Let's look at a sample saved_model TF model. 

In [1]:
!aws s3 cp s3://sagemaker-sample-data-us-east-1/tensorflow/model/resnet/resnet_50_v2_fp32_NCHW.tar.gz ./

download: s3://sagemaker-sample-data-us-east-1/tensorflow/model/resnet/resnet_50_v2_fp32_NCHW.tar.gz to ./resnet_50_v2_fp32_NCHW.tar.gz


In [2]:
!tar xvf resnet_50_v2_fp32_NCHW.tar.gz

tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
export/
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
export/._.DS_Store
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
export/.DS_Store
tar: Ignoring unknown extended header keyword `SCHILY.dev'
tar: Ignoring unknown extended header keyword `SCHILY.ino'
tar: Ignoring unknown extended header keyword `SCHILY.nlink'
export/._Servo
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Igno

In [3]:
!ls export/Servo/1527887769/

saved_model.pb	variables


In [4]:
!rm -r export/

### Getting a pretrained NGC model in Saved model format

In [5]:
import time
path = 'export/Servo/'+str(int(float(time.time())))+'/'
!mkdir -p {path}
!wget https://api.ngc.nvidia.com/v2/models/nvidia/rntf_fp16/versions/1/files/SavedModel/saved_model.pb -P {path}
!wget https://api.ngc.nvidia.com/v2/models/nvidia/rntf_fp16/versions/1/files/SavedModel/variables/variables.data-00001-of-00002 -P {path+'variables/'}
!wget https://api.ngc.nvidia.com/v2/models/nvidia/rntf_fp16/versions/1/files/SavedModel/variables/variables.data-00000-of-00002 -P {path+'variables/'}
!wget https://api.ngc.nvidia.com/v2/models/nvidia/rntf_fp16/versions/1/files/SavedModel/variables/variables.index -P {path+'variables/'}    
!tar -czvf model.tar.gz export/

--2020-04-24 14:59:19--  https://api.ngc.nvidia.com/v2/models/nvidia/rntf_fp16/versions/1/files/SavedModel/saved_model.pb
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 52.27.83.248, 54.71.37.245
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|52.27.83.248|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://s3.us-west-2.amazonaws.com/prod-model-registry-ngc-bucket/org/nvidia/models/rntf_fp16/versions/1/files/SavedModel/saved_model.pb?response-content-disposition=attachment%3B%20filename%3D%22saved_model.pb%22&response-content-type=application%2Foctet-stream&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEE8aCXVzLXdlc3QtMiJHMEUCIQDmeJ%2B6ds8gHjE0G5sdiDfpfFX7gNAHfCGWLS5Syz%2BFWQIgR%2FDSgJiYTL68n0AmUTPGrwC4qQ7YVkuhv3Bqok%2FzReUqtAMIeBACGgw3ODkzNjMxMzUwMjciDOpVGYVlMn8ZeKvJfCqRAwWx2TVMqnEJRuKDBbo7WPenSZraDt6EW0gR076vEvFgZ%2FZCBMJrSXOzEfh0RFy8l%2B%2BjcmPjKSD%2BtyuRFDpvQF%2FPu2k1tS4j9MttRy6m0wOULl1I11286lVxY4n8Xq3lQ50DNj%2BMWEK6hTK7LwvLnCuz6cye7djVC6nHpdFymgWZg

Connecting to s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)|52.218.221.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11990 (12K) [application/octet-stream]
Saving to: ‘export/Servo/1587740359/variables/variables.index’

variables.index     100%[===================>]  11.71K  --.-KB/s    in 0s      

2020-04-24 14:59:28 (126 MB/s) - ‘export/Servo/1587740359/variables/variables.index’ saved [11990/11990]

export/
export/Servo/
export/Servo/1587740359/
export/Servo/1587740359/saved_model.pb
export/Servo/1587740359/variables/
export/Servo/1587740359/variables/variables.index
export/Servo/1587740359/variables/variables.data-00001-of-00002
export/Servo/1587740359/variables/variables.data-00000-of-00002


In [6]:
!tar -xvf model.tar.gz

export/
export/Servo/
export/Servo/1587740359/
export/Servo/1587740359/saved_model.pb
export/Servo/1587740359/variables/
export/Servo/1587740359/variables/variables.index
export/Servo/1587740359/variables/variables.data-00001-of-00002
export/Servo/1587740359/variables/variables.data-00000-of-00002


In [7]:
!rm -r export/

## Setup role for SageMaker

Let's start by creating a SageMaker session and specifying the IAM role arn used to give hosting access to your model. See the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [8]:
import sagemaker

role = sagemaker.get_execution_role()

In [9]:
%%time
import boto3

# use the region-specific saved model object
region = boto3.Session().region_name
saved_model = sagemaker.session.Session().upload_data('model.tar.gz')

CPU times: user 1.96 s, sys: 301 ms, total: 2.26 s
Wall time: 1.76 s


## Load the TensorFlow Serving Model on Amazon SageMaker using Python SDK

We can use the SageMaker Python SDK to load our pre-trained TensorFlow Serving model for hosting in SageMaker for predictions.

There are a few parameters that our TensorFlow Serving Model is expecting.
1. `model_data` - The S3 location of a model tar.gz file to load in SageMaker
2. `role` - An IAM role name or ARN for SageMaker to access AWS resources on your behalf.
3. `framework_version` - TensorFlow Serving version you want to use for handling your inference request .


In [10]:
from sagemaker.tensorflow.serving import Model

tensorflow_model = Model(model_data=saved_model,
                         role=role,
                         framework_version='1.14')

# Deploy the trained Model to an Endpoint with an attached EI accelerator

The `deploy()` method creates an endpoint which serves prediction requests in real-time.

The only change required for utilizing EI with our SageMaker TensorFlow Serving containers only requires providing an `accelerator_type` parameter, which determines which type of EI accelerator to attach to your endpoint. The supported types of accelerators can be found here: https://aws.amazon.com/sagemaker/pricing/instance-types/


In [13]:
%%time
predictor = tensorflow_model.deploy(initial_instance_count=1,
                                    instance_type='ml.p3.2xlarge')

-----------------!CPU times: user 339 ms, sys: 0 ns, total: 339 ms
Wall time: 8min 32s


#  Invoke the Endpoint to get inferences

Invoking prediction:

In [14]:
import numpy as np
random_input = np.random.rand(1, 1, 3, 3)
prediction = predictor.predict({'inputs': random_input.tolist()})
print(prediction)

{u'outputs': {u'probabilities': [[0.00181863387, 0.000308345247, 0.000224663949, 0.000105218518, 0.000201158255, 0.000512797676, 0.000178170303, 0.00062313606, 0.000490190112, 0.000882363, 0.000362244697, 0.000480229326, 0.00047326702, 0.00022533437, 0.000501258473, 0.000778052257, 0.000779272639, 0.000122643818, 0.000330245733, 0.000326354726, 0.000250201731, 0.000135383525, 0.000283011468, 0.000219680092, 0.000612412, 0.000347099616, 0.00111330161, 0.000182586664, 0.00047361842, 0.000113796239, 0.000702265417, 0.00140311127, 0.000122181867, 0.000335647346, 0.000557845749, 0.000111529356, 0.000558486965, 0.00181321765, 0.00127990067, 0.000611173688, 0.00117498066, 0.000473144464, 0.000119554119, 0.000244195311, 0.000650275324, 0.000232503677, 0.000214546322, 0.000421376608, 0.00202968135, 0.000307032082, 0.000564544869, 0.000623003172, 0.000582267, 0.000890947354, 0.00105615589, 0.000241011177, 0.000209132821, 0.000372110924, 0.00046954, 0.000214680564, 0.00297602499, 0.00146043918, 0

# Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
print(predictor.endpoint)

In [ ]:
import sagemaker

predictor.delete_endpoint()